In [1]:
import numpy as np
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaModel
from torch.optim import Adam, lr_scheduler
import torch.nn.functional as F
import sys
sys.path.append('../')

import os
import argparse
import json
import torch.nn as nn

from util import *
from losses import LabelSmoothingCrossEntropy, SupConLoss, FocalLoss
from augment import *

from torch.utils.data.dataset import ConcatDataset
# from torch_model import SupConRobertaNet, SupConMultiRobertaNet
from torch.utils.data.sampler import RandomSampler
from torch_model import *
from preprop import *
from transformers import PreTrainedModel, BertPreTrainedModel
from transformers import (
        BertPreTrainedModel, BertModel, BertForPreTraining, BertForMaskedLM, BertLMHeadModel,
        BertForNextSentencePrediction, BertForSequenceClassification, BertForMultipleChoice,
        BertForTokenClassification, BertForQuestionAnswering,
        load_tf_weights_in_bert, BERT_PRETRAINED_MODEL_ARCHIVE_LIST,
        BertLayer, BertSelfAttention, BertAttention, BertLayer, BertEncoder
)

In [2]:
# !pip3 install pandas

In [3]:
class PetDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.SE_index = [ i for i, c in enumerate(df.columns) if "SE" in c][0]
        self.label_index = [ i for i, c in enumerate(df.columns) if "label_id" in c][0]
        self.Num_class = len(df[df.columns[self.label_index]].value_counts())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, self.SE_index]
        label = self.df.iloc[idx, self.label_index]
        return text, label
    

In [4]:
BATCH_SIZE = 8
MAX_SEQ_LEN = 512

In [5]:
train_df = pd.read_csv('files/train2.csv')
test_df = pd.read_csv('files/test2.csv')
# train_df = pd.read_csv('data/files/disease_train.csv')
# test_df = pd.read_csv('data/files/disease_test.csv')
train_df.SE = train_df.SE.apply(lambda x : preprocess(str(x)))
test_df.SE = test_df.SE.apply(lambda x : preprocess(str(x)))
Num_Label = len(train_df.label_id.value_counts())
print(Num_Label)
print(train_df.shape)
print(test_df.shape)

10
(4280, 5)
(476, 5)


In [6]:
pretrained_path = './pretrained_local/'
tokenizer = RobertaTokenizer.from_pretrained(pretrained_path, do_lower_case=False)

In [7]:
tokenizer.all_special_ids

[0, 4, 5, 4, 3, 0, 6]

In [8]:
tokenizer.convert_ids_to_tokens(0)

'<s>'

In [9]:
tokenizer.convert_ids_to_tokens(1)

'<s1>'

In [10]:
tokenizer.convert_ids_to_tokens(2)

'<s2>'

In [11]:
text = '안녕 반가워, hi my pet is good'

print(tokenizer.encode(text, add_special_tokens=False, max_length=15, truncation=True))
# print(tokenizer.encode(text, add_special_tokens=False, max_length=10, truncation=True))


[2500, 9570, 1259, 1149, 2835, 18, 22284, 6948, 15695, 1507, 3686]


In [12]:
e = [2500, 9570, 1259, 1149, 2835, 18, 22284, 6948, 15695, 1507, 3686]
print(e[:20] + [0] * (20-len(e[:20])))

[2500, 9570, 1259, 1149, 2835, 18, 22284, 6948, 15695, 1507, 3686, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
print(tokenizer.encode(text, add_special_tokens=False, max_length=10-2, truncation=True))
print(tokenizer.encode(text, add_special_tokens=False, max_length=10-2, truncation=True))

[2500, 9570, 1259, 1149, 2835, 18, 22284, 6948]
[2500, 9570, 1259, 1149, 2835, 18, 22284, 6948]


In [14]:
tokenized = tokenizer.encode(text, add_special_tokens=False, max_length=10-4, truncation=True)
# tokenized.append(4)
# tokenized.insert(0,2)
# tokenized.insert(0,1)
# tokenized.insert(0,0)
tokenized = [0,1,2] + tokenized + [4]
print(tokenized)

[0, 1, 2, 2500, 9570, 1259, 1149, 2835, 18, 4]


In [15]:
device = torch.device("cuda")
# device = torch.device('cpu')
# pretrained_path = './pretrained_without_wiki'
pretrained_path = './pretrained_local/'
# pretrained_path = './pretrained_0824/'
tokenizer = RobertaTokenizer.from_pretrained(pretrained_path, do_lower_case=False)
# model = YuBertNet(path=pretrained_path,                       
#                               embedding_dim=768,
#                               num_class=Num_Label)

model = YuBertNet.from_pretrained(pretrained_path, num_labels=Num_Label)
# model = YubertForSequenceClassification.from_pretrained(pretrained_path, num_labels=Num_Label)
model.to(device)
# criterion = torch.nn.CrossEntropyLoss()
# criterion = LabelSmoothingCrossEntropy()
# criterion = SupConLoss()
# criterion = criterion.to(device)

Some weights of the model checkpoint at ./pretrained_local/ were not used when initializing YuBertNet: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing YuBertNet from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing YuBertNet from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of YuBertNet were not initialized from the model checkpoint at ./pretrained_local/ and are newly initialized: ['down_fc.0.weight', 'down_fc.0.bias', 'down_fc.2.weight', 'down_fc.2.bias']
You should probably TRAIN this model on a down-stream task to be able to us

YuBertNet(
  (roberta): YubertModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 384, padding_idx=1)
      (position_embeddings): Embedding(516, 384, padding_idx=1)
      (token_type_embeddings): Embedding(1, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): YuBertEncoder(
      (layer): ModuleList(
        (0): YuBertLayer(
          (attention): YuBertAttention(
            (self): YuBertSelfAttentionJupyter(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), 

In [16]:
# def reset_parameters(model):
#     for p in model.parameters():
#         if p.dim() > 1:
#             nn.init.xavier_uniform_(p)
            
# reset_parameters(model)            

In [17]:
# for param in model.parameters() :
#     print(param)

In [18]:
# for param, state in zip(model.parameters(), model.state_dict()) :
#     print(state)
#     print(param.size())

In [19]:
def model_eval(test_df) :
#     device = torch.device("cuda")
#     model.to(device)       
    model.eval()

    test_dataset = PetDataset(test_df)
    test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=2)

    total_loss = 0
    total_len = 0
    total_correct = 0

    for text, label in test_loader:
        #   encoded_list = [tokenizer.encode(t, add_special_token=True) for t in text]
        encoded_list = [tokenizer.encode(t, max_length=512, truncation=True) for t in text]
        padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        label = torch.tensor(label)
        outputs = model(sample=sample)
        logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(label)
        total_correct += correct.sum().item()
        total_len += len(label)

    print('Test accuracy: ', total_correct / total_len) 

In [20]:
# for param, state in zip(model.parameters(), model.state_dict()) :
#     print(state)
#     print(param.size())
# for param, state in zip(model.parameters(), model.state_dict()) :
#     print(state)
#     print(param)

In [21]:
train_dataset = PetDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [22]:
# # Learning_rate=0.00001
# Learning_rate=0.00001
# optimizer = Adam(model.parameters(), lr=Learning_rate)
# scheduler = lr_scheduler.LambdaLR(
#     optimizer=optimizer, lr_lambda=lambda epoch: 1 / ((epoch/100)+1)
# )

In [23]:
# model.load_state_dict(torch.load('yubert/tune1'))

In [24]:
# torch.save(model.state_dict(), 'yubert/tune1')

In [25]:
# model.load_state_dict(torch.load('yubert/tune1'))

In [26]:
# for param, state in zip(model.parameters(), model.state_dict()) :
#     if 'fc.' not in state :
#         param.requires_grad = False


In [27]:
# for param, state in zip(model.parameters(), model.state_dict()) :
#     torch.nn.init.normal_(param, mean=0, std=1)
# #     print(param)

In [28]:
optimizer = Adam(model.parameters(), lr=0.00001)
scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / ((epoch/4) + 1)
)

In [29]:
# model.load_state_dict(torch.load('contrastive/tune5'))

In [30]:
# # optimizer = Adam(model.parameters(), lr=0.00008)

# # scheduler = lr_scheduler.LambdaLR(
# #     optimizer=optimizer, lr_lambda=lambda epoch: 1 / (int(epoch/3) + 1)
# # )

# epochs = 10
# model.train()

# for epoch in range(epochs):
#     total_loss = 0
#     total_len = 0
#     total_correct = 0
#     total_count = 0
#     model.train()
#     for text, label in train_loader:

#         encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=512, truncation=True) for t in text]
#         padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
#         sample = torch.tensor(padded_list)
#         sample, label = sample.to(device), label.to(device)
#         labels = torch.tensor(label)
#         outputs = model(sample, labels=labels)
#         loss, logits = outputs

#         pred = torch.argmax(F.softmax(logits), dim=1)
#         correct = pred.eq(labels)
#         total_correct += correct.sum().item()
#         total_len += len(labels)
#         total_loss += loss.item()
#         total_count += 1

#         optimizer.zero_grad()    
#         loss.backward()
#         optimizer.step()
#     scheduler.step()
#     model_eval(test_df)

#     print('[Epoch {}/{}] Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, epochs, total_loss / total_count, total_correct / total_len))

In [31]:
def tokenize(text : str)  : 
    tokenized_list = tokenizer.encode(text, add_special_tokens=True, max_length=512-2, truncation=True)
    tokenized_list = [0, 1, 2] + tokenized_list[1:-1] + [4]
    return tokenized_list

In [ ]:
CUDA_LAUNCH_BLOCKING=1
model.train()
criterion = FocalLoss(alpha=0.97, reduce=True)
# criterion = torch.nn.CrossEntropyLoss()
criterion.to(device)
epochs = 20

for epoch in range(epochs):
    losses = AverageMeter()
    total_loss = 0
    total_len = 0
    total_correct = 0
    total_count = 0
    model.train()
    for text, label in train_loader:
#         print(text)
#         encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=512, truncation=True) for t in text]
        encoded_list = [tokenize(t) for t in text]
#         print(encoded_list)
        padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        label = torch.tensor(label)
        outputs = model(sample=sample)

        loss = criterion(outputs, label)
        losses.update(loss.item(), BATCH_SIZE)
        pred = torch.argmax(F.softmax(outputs), dim=1)
        correct = pred.eq(label)
        total_correct += correct.sum().item()
        total_len += len(label)
        total_loss += loss.item()
        total_count += 1
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 

#         if (total_count + 1) % 200 == 0:
# #             print_weight(model)
#             print('Train: [{0}][{1}/{2}]\t'
#                   'loss {loss.avg: .5f}'.format(
#                    epoch, total_count + 1, len(train_loader), loss=losses))   

    model.train()
    scheduler.step()
    model_eval(test_df)
    print('[Epoch {}/{}] Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, epochs, total_loss / total_count, total_correct / total_len))
#     print(tloss)  

/home/beomgon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/beomgon/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/home/beomgon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/beomgon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/beomgon/.local/lib/python3.6/site-packages/i

Test accuracy:  0.1407563025210084
[Epoch 1/20] Train Loss: 1.7780, Accuracy: 0.139
Test accuracy:  0.2710084033613445
[Epoch 2/20] Train Loss: 1.6626, Accuracy: 0.206
Test accuracy:  0.40336134453781514
[Epoch 3/20] Train Loss: 1.2260, Accuracy: 0.380
Test accuracy:  0.5210084033613446
[Epoch 4/20] Train Loss: 0.8980, Accuracy: 0.493
Test accuracy:  0.5840336134453782
[Epoch 5/20] Train Loss: 0.6369, Accuracy: 0.622
Test accuracy:  0.5756302521008403
[Epoch 6/20] Train Loss: 0.4747, Accuracy: 0.709
Test accuracy:  0.6869747899159664
[Epoch 7/20] Train Loss: 0.3771, Accuracy: 0.764
Test accuracy:  0.6743697478991597
[Epoch 8/20] Train Loss: 0.3090, Accuracy: 0.799


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'contrastive/tune5')

In [ ]:
# for param, state in zip(model.parameters(), model.state_dict()) :
#     if 'fc.' not in state :
#         param.requires_grad = True
#     else  :
#         param.requires_grad = False        

In [ ]:
# optimizer = Adam(model.parameters(), lr=0.00001)
# scheduler = lr_scheduler.LambdaLR(
#     optimizer=optimizer, lr_lambda=lambda epoch: 1 / ((epoch/2) + 1)
# )

In [ ]:
# model.train()
# criterion = FocalLoss(alpha=0.97, reduce=True)
# # criterion = torch.nn.CrossEntropyLoss()
# criterion.to(device)
# epochs = 7

# for epoch in range(epochs):
#     losses = AverageMeter()
#     total_loss = 0
#     total_len = 0
#     total_correct = 0
#     total_count = 0
#     model.train()
#     for text, label in train_loader:
# #         print(label)
#         encoded_list = [tokenizer.encode(t, add_special_tokens=True, max_length=512, truncation=True) for t in text]
#         padded_list = [e[:512] + [0] * (512-len(e[:512])) for e in encoded_list]
#         sample = torch.tensor(padded_list)
#         sample, label = sample.to(device), label.to(device)
#         label = torch.tensor(label)
#         outputs = model(sample=sample, iscontra=False)

#         loss = criterion(outputs, label)
#         losses.update(loss.item(), BATCH_SIZE)
# #         print(loss)
        
# #         total_correct += correct.sum().item()
#         total_len += len(label)
#         total_loss += loss.item()
#         total_count += 1
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step() 

#         if (total_count + 1) % 200 == 0:
# #             print_weight(model)
#             print('Train: [{0}][{1}/{2}]\t'
#                   'loss {loss.avg:.5f}'.format(
#                    epoch, total_count + 1, len(train_loader), loss=losses))   

#     model.train()
#     scheduler.step()
#     model_eval(test_df)
#     print('***********************************')
# #     print(tloss)  